## Whole dataset 5k genes
For the integration we needed the top 2k hvg, this filters out lots of markers that might be of interest, so we keep here atleast the top 5k. (can also keep all genes, just takes more memory)

In [1]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/home/hers_basak/jjiang/jack/miniconda/envs/scarches2/bin/python
3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
sys.version_info(major=3, minor=9, micro=18, releaselevel='final', serial=0)


In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))#embed HTML within IPython notebook and make containers 90% of screen
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import seaborn as sns
import os
import matplotlib.pyplot as plt
#import scvi
import torch

%matplotlib inline
sc.settings.verbosity = 0             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, facecolor='white') #figure resolution and background color

#To make reproducible plots
np.random.seed(41)
# This is used
# os.path.join('data/'+object_names[i][6:10]+'_preprocessed.h5ad')
#sc.logging.print_versions()

In [3]:
#set data path
os.chdir('/home/hers_basak/jjiang/jack/outputs/deliverables/5_integration/benchmark/')

Need 2 files:
- Base: .h5ad  file with all the genes/ the amount of genes you want.
- Annotation: .h5ad file with annotation and integration latent space.

We will be adding the annotations from the integrated to the base.

In [31]:
base= sc.read_h5ad('/home/hers_basak/jjiang/jack/outputs/deliverables/5_integration/data/complete_base.h5ad')

In [48]:
annotation=sc.read_h5ad('/home/hers_basak/jjiang/jack/outputs/deliverables/5_integration/data/scvi/models/scvi_model_final/adata_predicted_lowest_filtered.h5ad')

In [50]:
base

AnnData object with n_obs × n_vars = 522220 × 2000
    obs: 'species', 'gender', 'age', 'instrument', 'technology', 'atlas', 'sample', 'batch_1', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'n_genes', 'total_counts_mito', 'total_counts_norm', 'original_Library', 'original_Brain_Region', 'original_Level_1_cell_type', 'original_Level_2_cell_type', 'original_cellclass_lvl1', 'original_subclass_lvl3_1', 'original_cellclass_lvl1_n', 'original_lineage', 'original_subclass_lvl2', 'original_celltype_lvl3', 'donor', 'original_supercluster_term', 'original_cluster_id', 'original_subcluster_id', 'original_celltype', 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', '_scvi_batch', '_scvi_labels', 'leiden', 'cell_type_low_res', 'cell_type_high_res', 'harmonized_cell_type', 'harmonized_cell_

In [49]:
annotation

AnnData object with n_obs × n_vars = 507763 × 2000
    obs: 'species', 'gender', 'age', 'instrument', 'technology', 'atlas', 'sample', 'batch_1', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'n_genes', 'total_counts_mito', 'total_counts_norm', 'original_Library', 'original_Brain_Region', 'original_Level_1_cell_type', 'original_Level_2_cell_type', 'original_cellclass_lvl1', 'original_subclass_lvl3_1', 'original_cellclass_lvl1_n', 'original_lineage', 'original_subclass_lvl2', 'original_celltype_lvl3', 'donor', 'original_supercluster_term', 'original_cluster_id', 'original_subcluster_id', 'original_celltype', 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', '_scvi_batch', '_scvi_labels', 'leiden', 'cell_type_low_res', 'cell_type_high_res', 'harmonized_cell_type', 'harmonized_lowes

The first problem: the annotation file labels are not harmonized.

Second problem: the base file has too many observations that were filtered out later on. We need to filter out all cells that are not in the annotation file.

In [ ]:
base_filtered=base[base.obs_names.isin(annotation.obs_names)]

Then we can transfer the predicted labels

In [62]:
base_filtered.obs['harmonized_lowest_filtered_02_scvi']=annotation.obs['harmonized_lowest_filtered_02_scvi']

In [64]:
base

View of AnnData object with n_obs × n_vars = 507763 × 45433
    obs: 'species', 'gender', 'age', 'instrument', 'technology', 'atlas', 'sample', 'batch_1', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'n_genes', 'total_counts_mito', 'total_counts_norm', 'original_Library', 'original_Brain_Region', 'original_Level_1_cell_type', 'original_Level_2_cell_type', 'original_cellclass_lvl1', 'original_subclass_lvl3_1', 'original_cellclass_lvl1_n', 'original_lineage', 'original_subclass_lvl2', 'original_celltype_lvl3', 'donor', 'original_supercluster_term', 'original_cluster_id', 'original_subcluster_id', 'original_celltype', 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'harmonized_lowest_filtered_02_scvi'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'

Or any other annotation that might be interesting.

In [66]:
for i in ['cell_type_low_res', 'cell_type_high_res', 'cell_type_lowest_res', 'cell_type_med_res','harmonized_cell_type', 'harmonized_lowest_filtered_02_scvi',  'harmonized_lowest', 'harmonized_lowest_predicted']:
    base.obs[i]=annotation.obs[i]

In [67]:
base.obsm['X_scVI']=annotation.obsm['X_scVI']

Depending on what you want to do, you can take the top N hvg.

In [65]:
sc.pp.highly_variable_genes(
    base,
    n_top_genes=5000,
    batch_key="atlas",
    subset=True)

/home/hers_basak/jjiang/jack/miniconda/envs/scarches2/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/home/hers_basak/jjiang/jack/miniconda/envs/scarches2/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/home/hers_basak/jjiang/jack/miniconda/envs/scarches2/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:215: FutureWarning: T

In [68]:
base.write('/home/hers_basak/jjiang/jack/outputs/deliverables/5_integration/data/scvi/models/scvi_model_final/adata_predicted_lowest_filtered_5k.h5ad')